importing packages...

In [2]:
from pathlib import Path
import json
import os
import xml.etree.ElementTree as ET
from xml.dom import minidom
import csv


Creating XML files in "XML" folder(manually change the path->thinking of integrating in one folder with renaming later)

In [6]:
import os
import json
import xml.etree.ElementTree as ET
from xml.dom import minidom
from pathlib import Path

def create_retinanet_xml(annotation_data, json_filename_without_extension):
    annotation = ET.Element('annotation')
    
    image_info = ET.SubElement(annotation, 'image')
    filename = ET.SubElement(image_info, 'filename')
    filename.text = json_filename_without_extension + '.png'  # Add .jpg extension
    
    size = ET.SubElement(image_info, 'size')
    width = ET.SubElement(size, 'width')
    height = ET.SubElement(size, 'height')
    
    width.text = str(annotation_data['imageWidth'])
    height.text = str(annotation_data['imageHeight'])
    
    objects = ET.SubElement(annotation, 'objects')
    
    for shape in annotation_data['shapes']:
        object_info = ET.SubElement(objects, 'object')
        
        label = ET.SubElement(object_info, 'label')
        label.text = shape['label']
        
        points = shape['points']
        x_min = min(points[0][0], points[1][0])
        y_min = min(points[0][1], points[1][1])
        x_max = max(points[0][0], points[1][0])
        y_max = max(points[0][1], points[1][1])
        
        bbox = ET.SubElement(object_info, 'bbox')
        x_min_elem = ET.SubElement(bbox, 'xmin')
        y_min_elem = ET.SubElement(bbox, 'ymin')
        x_max_elem = ET.SubElement(bbox, 'xmax')
        y_max_elem = ET.SubElement(bbox, 'ymax')
        
        x_min_elem.text = str(x_min)
        y_min_elem.text = str(y_min)
        x_max_elem.text = str(x_max)
        y_max_elem.text = str(y_max)
    
    return annotation

json_folder_path = Path("/home/aihub_juhoon/keras-retinanet-main/train_data/json")
xml_folder_path = Path("/home/aihub_juhoon/keras-retinanet-main/train_data/XML")
os.makedirs(xml_folder_path, exist_ok=True)

for file_name in os.listdir(json_folder_path):
    if file_name.endswith('.json'):
        json_file_path = os.path.join(json_folder_path, file_name)
        json_filename_without_extension = os.path.splitext(file_name)[0]  # JSON filename without extension
        xml_filename = json_filename_without_extension + '.xml'
        xml_file_path = os.path.join(xml_folder_path, xml_filename)
        
        # Load JSON annotations
        with open(json_file_path, 'r') as json_file:
            annotations = json.load(json_file)
        
        # Create the XML tree
        xml_tree = ET.ElementTree(create_retinanet_xml(annotations, json_filename_without_extension))
        
        # Save the XML file with pretty formatting
        xml_tree.write(xml_file_path, encoding='utf-8', xml_declaration=True)
        xml_string = minidom.parseString(ET.tostring(xml_tree.getroot())).toprettyxml(indent="  ")
        with open(xml_file_path, "w") as xml_file:
            xml_file.write(xml_string)
        
        print(f"Converted {file_name} to RetinaNet XML format and saved as {xml_filename}")


Converted 1102_extract_frame3_000069.json to RetinaNet XML format and saved as 1102_extract_frame3_000069.xml
Converted extract_frame7_000022.json to RetinaNet XML format and saved as extract_frame7_000022.xml
Converted extract_frame2_000080.json to RetinaNet XML format and saved as extract_frame2_000080.xml
Converted extract_frame2_000067.json to RetinaNet XML format and saved as extract_frame2_000067.xml
Converted extract_frame3_000055.json to RetinaNet XML format and saved as extract_frame3_000055.xml
Converted extract_frame2_000011.json to RetinaNet XML format and saved as extract_frame2_000011.xml
Converted 1102_extract_frame4_000074.json to RetinaNet XML format and saved as 1102_extract_frame4_000074.xml
Converted extract_frame3_000020.json to RetinaNet XML format and saved as extract_frame3_000020.xml
Converted extract_frame2_000003.json to RetinaNet XML format and saved as extract_frame2_000003.xml
Converted extract_frame1_000048.json to RetinaNet XML format and saved as extrac

KeyboardInterrupt: 

XML files to one csv file with label datas....change the path as you need

In [4]:
import os
import xml.etree.ElementTree as ET
import csv

# Input: Path to the directory containing XML files
xml_directory = '/home/aihub_juhoon/keras-retinanet-main/train_data/XML'  # Add the path to your XML files directory
image_folder = '/home/aihub_juhoon/keras-retinanet-main/train_data/data'  # Add the path to your image folder
csv_save_path = '/home/aihub_juhoon/keras-retinanet-main/train_data/annotation.csv'

# Check if the CSV file already exists
if not os.path.exists(csv_save_path):
    # Create the CSV file and write the header
    with open(csv_save_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        

# Open CSV file for writing
with open(csv_save_path, 'a', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Process each XML file in the directory
    for filename in os.listdir(xml_directory):
        if filename.endswith('.xml'):
            xml_path = os.path.join(xml_directory, filename)
            
            tree = ET.parse(xml_path)
            root = tree.getroot()
            
            image_filename = root.find('image/filename').text
            full_image_path = os.path.join(image_folder, image_filename)
            
            # Process each object in the XML
            for obj in root.findall('objects/object'):
                xmin1 = round(float(obj.find('bbox/xmin').text), 9)
                ymin1 = round(float(obj.find('bbox/ymin').text), 9)
                xmax1 = round(float(obj.find('bbox/xmax').text), 9)
                ymax1 = round(float(obj.find('bbox/ymax').text), 9)
                x1=int(xmin1)
                y1=int(ymin1)
                x2=int(xmax1)
                y2=int(ymax1)
                label = obj.find('label').text
                
                # Write object data to CSV
                csv_writer.writerow([full_image_path, x1, y1, x2, y2, label])

print(f'Data written to {csv_save_path}')


Data written to /home/aihub_juhoon/keras-retinanet-main/train_data/annotation.csv


create classmapping.csv(eg. boat:0, end:1, ....) 

In [5]:
input_file_path = '/home/aihub_juhoon/keras-retinanet-main/train_data/annotation.csv'
output_file_path = '/home/aihub_juhoon/keras-retinanet-main/train_data/classmapping.csv'

categories = {}  

with open(input_file_path, 'r') as input_file:
    csv_reader = csv.reader(input_file)
    next(csv_reader)  # Skip the header row

    for row in csv_reader:
        category = row[5]  # Assuming the category is in the 6th column (index 5)
        if category not in categories:
            categories[category] = len(categories)  

with open(output_file_path, 'w', newline='') as output_file:
    csv_writer = csv.writer(output_file)

    for category, code in categories.items():
        csv_writer.writerow([category, code])

print("End")


End


referred blog:https://boysboy3.tistory.com/149

with tensorflow-gpu,and keras version it is outdated so need to find a match with one with python version